In [1]:
import wandb

api = wandb.Api()
runs = api.runs("eth-ticinesi/FORL_Briscola")

In [3]:
import pandas as pd
import os
os.makedirs('./checkpoints/pretrained', exist_ok=True)
commands = []

for run in runs:
    if 'PRETRAINED' not in run.tags:
        continue

    arch = 'nn' if 'NN' in run.tags else 'rnn'
    num = len(run.logged_artifacts())
    for artifact in run.logged_artifacts():
        if artifact.type != 'model':
            continue
      
        original_name = f"{artifact.name.split(':')[0]}.pth"

        if 'policy_train' in artifact.name:
            continue
        elif 'policy_last' in artifact.name:
            step = artifact.name.split('policy_last_')[1].split(':')[0]
            last = True
        
        artifact.download('./tmp/')
        path = f'{run.id}_base.pt'
        os.rename(f'./tmp/{original_name}', f'./checkpoints/pretrained/{path}')
        hidden_dim = run.config['hidden_dim'] if 'hidden_dim' in run.config else 64
        rnn_out = run.config['rnn_out_size'] if 'rnn_out_size' in run.config else 0
        
        if arch == 'rnn':
            cmd = f'python3 ./ppo_RNN_universal.py --track  --cuda True --exp-name 2PHASE_PPO_universal_COM_RNN_h={hidden_dim}_rnn={rnn_out}  \
                        --sample-batch-env False --hidden-dim {hidden_dim} --rnn-out-size {rnn_out} --briscola-communicate --resume-path ./checkpoints/pretrained/{path} \
                            --briscola-communicate-truth False --total-timesteps {12*8*1000000} --freq-save-model 10000'
        else:
            s = run.config["sample_batch_env"]
            suffix = '_more_samples' if not s else ''
            cmd = f'python3 ./ppo_com_universal.py --track  --cuda False --exp-name 2PHASE_PPO_universal_COM_RNN_h={hidden_dim}{suffix}  \
                        --sample-batch-env {s} --hidden-dim {hidden_dim} --briscola-communicate --resume-path ./checkpoints/pretrained/{path} \
                            --briscola-communicate-truth False --total-timesteps {12*8*1000000} --freq-save-model 10000'
        commands.append(cmd)

with open('./experiment_2phase.txt', 'w') as f:
    for cmd in commands:
        f.write(cmd + '\n')

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
